<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN07/7.2-Callbacks_und_Tensorboard_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Callbacks und Tensorboard

This notebook contains the code samples found in Chapter 7, Section 2 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). 

Die Themen in diesem Kapitel:
- **PyTorch Lightning Callbacks** verwenden
- Das Visualisierungstool **TensorBoard**
- Bewährte Verfahren für die Entwicklung von Modellen nach dem **aktuellen Stand der Technik**

In dieser Lektion lernen wir eine Reihe leistungsfähiger Tools kennen, die es uns erleichtern, schwierige Aufgaben Modelle nach dem aktuellen Stand der Technik zu entwickeln. 
- PyTorch Lightning Callbacks und das browserbasierte Visualisierungstool TensorBoard ermöglichen es, Modelle während des Trainings zu überwachen.
- Darüber hinaus kommen verschiedene andere bewährte Verfahren zur Sprache, wie die Normierung von Stapeln (*batch normalization*), residuale Verbindungen (*residual connections*), Hyperparameteroptimierung (*hyperparameter tuning*) und Ensemblemodelle (*ensemble models*).

### 7.1 Deep-Learning-Modelle mit PyTorch Lightning und TensorBoard untersuchen und überwachen

In diesem Abschnitt werden wir die Möglichkeiten untersuchen, mehr Kontrolle
darüber zu erlangen, was beim Training eines Modells geschieht. 
- Das Training eines Modells mit einer grossen Datenmenge über Dutzende Epochen hinweg durch den Aufruf von `trainer.fit()` zu starten, hat eine gewisse Ähnlichkeit mit dem Werfen eines Papierflugzeugs: Nachdem Sie es losgelassen haben, können Sie keinen Einfluss mehr auf die Flugbahn oder den Ort der Landung nehmen. 
- Wenn Sie verhindern möchten, dass die Flüge ein schlimmes Ende nehmen (und die Papierflugzeuge zerstört werden), wäre es besser, statt eines Papierflugzeugs eine Drohne zu verwenden, die ihre Umgebung wahrnimmt, Daten an den Drohnenpiloten übermittelt und automatisch auf den aktuellen Zustand reagieren kann. 
- Der Aufruf von `trainer.fit()` macht mithilfe der hier vorgestellten Verfahren aus einem Papierflugzeug eine intelligente autonome Drohne, die sich selbst überwacht und dynamisch auf Ereignisse reagieren kann.

### Beeinflussung eines Modells während des Trainings durch Callbacks

Beim Trainieren eines Modells gibt es viele Dinge, die sich nicht vorhersagen lassen.
Sie wissen insbesondere nicht, wie viele Epochen erforderlich sind, um bei
der Validierung den optimalen Wert der Verlustfunktion zu erreichen. In den bisherigen
Beispielen haben wir die Strategie verfolgt, das Modell so lange zu trainieren,
bis eine Überanpassung einsetzt. Anschließend wurde das Modell von Grund
auf neu mit der so ermittelten Anzahl von Epochen trainiert. Aber dieser Ansatz
ist natürlich aufwendig und unwirtschaftlich. 


### Callbacks

Besser wäre es, das Training abzubrechen, wenn Sie feststellen, dass sich die
Werte der Verlustfunktion nicht mehr verbessern, und genau das lässt sich mit
PyTorch Lightning's `Callbacks` erreichen. 
- Ein `Callback` ist ein Objekt (eine Klasseninstanz, die bestimmte Methoden implementiert), das dem Trainer beim Aufruf von `fit()` übergeben wird. 

- Anschließend ruft der Trainer dieses Objekt zu verschiedenen Zeitpunkten während des Trainings auf. 

- Das `Callback`-Objekt hat Zugriff auf alle verfügbaren Daten über den Zustand des Modells und dessen Leistung und kann Maßnahmen ergreifen: das Training unterbrechen, ein Modell speichern, andere Gewichtungen einlesen oder den Zustand des Modells auf andere Weise ändern.

### Anwendungen von Callbacks:

- Fixpunkterstellung (Model Checkpointing) – Die aktuellen Gewichtungen des Modells werden zu bestimmten Zeitpunkten während des Trainings automatisch gespeichert, z. B. das beste Modell basierend auf der Validierungsmetrik.
→ ``lightning.pytorch.callbacks.ModelCheckpoint``

- early stopping (früher Abbruch) – Wenn sich der Validierungsverlust oder eine andere Metrik über mehrere Epochen nicht mehr verbessert, wird das Training abgebrochen. Das beste gefundene Modell bleibt erhalten.
→ ``lightning.pytorch.callbacks.EarlyStopping``

- Dynamische Anpassung bestimmter Hyperparameter während des Trainings – typischerweise die Lernrate des Optimierers.
→ z. B. ``torch.optim.lr_scheduler.LambdaLR`` oder ``ReduceLROnPlateau`` über ``configure_optimizers``

- Protokollierung von Leistungskennzahlen des Trainings und der Validierung oder auch Visualisierung von Metriken und erlernten Repräsentationen während des Trainings.
→ ``lightning.pytorch.loggers.CSVLogger``, TensorBoardLogger, etc.

Der bekannte Fortschrittsbalken beim Training kommt auch in Lightning – standardmässig – über einen eingebauten Callback (ProgressBar).
| Lightning Entsprechung                                     |
|------------------------------------------------------------|
| `lightning.pytorch.callbacks.ModelCheckpoint`              |
| `lightning.pytorch.callbacks.EarlyStopping`                |
| `torch.optim.lr_scheduler.LambdaLR` + `configure_optimizers` |
| `torch.optim.lr_scheduler.ReduceLROnPlateau` + `configure_optimizers` |
| `lightning.pytorch.loggers.CSVLogger`                      |


Einige davon werden wir etwas näher betrachten, damit Sie eine Vorstellung
davon bekommen, wie man sie verwendet, nämlich 
- `ModelCheckpoint` (Fixpunkterstellung),
- `EarlyStopping` (früher Abbruch) und 
- `ReduceLROnPlateau` (Anpassung der Lernrate).
## 7.2 Ein bekanntes Beispiel (CNN, MNIST-Datensatz)


### Callbacks und TensorBoard mit PyTorch Lightning

In diesem Notebook lernst du, wie man mit **PyTorch Lightning** (importiert als `L`) das Training von Deep-Learning-Modellen überwacht und steuert.

**Zentrale Themen:**
- Verwendung von **Callbacks** zur dynamischen Beeinflussung des Trainings
- Visualisierung von Metriken mit **TensorBoard**
- Best Practices: Early Stopping, Model Checkpointing, Lernratenanpassung


In [ ]:
import torch
import numpy as np
import lightning.pytorch as L
import matplotlib.pyplot as plt

from torch import nn
from torchvision import datasets, transforms

from torch.utils.data import DataLoader, random_split


### Motivation: Kontrolle beim Modelltraining

Beim Training von Deep-Learning-Modellen laufen viele Prozesse automatisch ab. Ohne Kontrolle könnte das Training:
- zu lange dauern
- in Überanpassung (Overfitting) enden
- eine suboptimale Lernrate verwenden

✔ Durch den Einsatz von **Callbacks** können wir eingreifen:
- **Training frühzeitig abbrechen** (Early Stopping)
- **Bestes Modell speichern** (ModelCheckpoint)
- **Lernrate dynamisch anpassen** (ReduceLROnPlateau)

✔ Mit **TensorBoard** sehen wir, wie sich Metriken entwickeln, und können fundierte Entscheidungen treffen.


### PyTorch Lightning: Vorteile

Lightning strukturiert deinen Code und entlastet dich von Boilerplate:
- ➔ Trainings- und Validierungslogik übersichtlich in `training_step`, `validation_step`
- ➔ Einfaches Logging mit `self.log()`
- ➔ Integration von Callbacks und TensorBoard ohne Mehraufwand


### Daten laden (MNIST)



In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = datasets.MNIST(root="data", train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root="data", train=False, download=True, transform=transform)

train_ds, val_ds = random_split(mnist_train, [55000, 5000])
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128)
test_loader = DataLoader(mnist_test, batch_size=128)

# Visualisierung einiger Bilder
fig, axs = plt.subplots(5, 10, figsize=(12, 6))
k = 0
for i in range(5):
    for j in range(10):
        axs[i, j].imshow(train_ds[k][0].squeeze(), cmap="gray")
        axs[i, j].axis("off")
        k += 1
plt.show()


### Modell als LightningModule definieren

Ein `LightningModule` definiert:
- das Modell (Layers, Architektur)
- die Loss-Funktion
- die Schritte für Training und Validierung
- Optimizer und Scheduler


In [ ]:
class MNISTModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(64 * 5 * 5, 10),
            nn.LogSoftmax(dim=1),
        )
        self.loss_fn = nn.NLLLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, on_epoch=True)
        self.log("train_acc", acc, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
        }

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        acc = (y_hat.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss


### Callbacks erklärt

### Was genau machen Callbacks im Detail?
Was genau machen Callbacks im Detail?
- **ModelCheckpoint**
    - Speichert automatisch das Modell, wenn sich eine bestimmte Metrik verbessert (z. B. ``val_loss`` oder ``val_acc``).
    - Vorteil: Du verlierst nie das beste Modell – selbst wenn das Training später schlechter wird.
    - ➔ Beispiel: Bestes Ergebnis war bei Epoche 17 → dieses Modell wird gespeichert.
    - Pfad zur Datei: Standard oder via ``dirpath`` definierbar.

- **EarlyStopping**
    - Stoppt das Training automatisch, wenn sich eine Metrik über mehrere Epochen nicht mehr verbessert.
    - Verhindert Overfitting und spart Zeit.
    - ➔ Beispiel: ``patience=3`` ➔ nach 3 Epochen ohne Verbesserung wird gestoppt.
    - Das beste Modell bleibt gespeichert (in Kombi mit ModelCheckpoint).

- **TensorBoard – Visualisierung leicht gemacht**
    - Ein browserbasiertes Tool zur Überwachung des Trainingsverlaufs.
    - Zeigt:
        - 📊 Verlust (``Loss``) und Genauigkeit (``Accuracy``) über die Zeit
        - 🔍 Modellgraph (Architektur)
        - 📈 Lernratenverlauf
        - 📉 Gewichtshistogramme
        - 📝 Eigene Logs (Text, Bilder etc.)

[noch mehr Lightning Callbacks hier!](https://lightning.ai/docs/pytorch/stable/extensions/callbacks.html)

In [ ]:
checkpoint_cb = L.callbacks.ModelCheckpoint(
    monitor="val_loss", save_top_k=1, mode="min", filename="mnist-best"
)

earlystop_cb = L.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")

logger = L.loggers.TensorBoardLogger("tb_logs", name="mnist_model")


### Training starten


In [ ]:
model = MNISTModel()

trainer = L.Trainer(
    max_epochs=5,
    callbacks=[checkpoint_cb, earlystop_cb],
    logger=logger,
    accelerator="auto",
    devices="auto",
    log_every_n_steps=10,
)

trainer.fit(model, train_loader, val_loader)


### Model testen

In [ ]:
trainer.test(model, dataloaders=test_loader)


### TensorBoard starten in Kommandozeile

```bash
%tensorboard --logdir=Lektionen/ANN07/pytorchlightning/tb_logs
```

➔ Im Browser: `http://localhost:6006`

Du siehst: Trainingskurven, Validierungsmetriken, Histogramme, Graphen. (eventuell Pfad anpassen!)

### Benutzerdefinierter Callback: Aktivierungen speichern


In [ ]:
class ActivationLoggerCallback(L.Callback):
    def on_epoch_end(self, trainer, pl_module):
        sample = next(iter(val_loader))[0][0:1].to(pl_module.device)
        with torch.no_grad():
            activations = pl_module.model(sample)
        np.save(
            f"activations_epoch_{trainer.current_epoch}.npy", activations.cpu().numpy()
        )


# Training mit eigenem Callback
trainer_with_custom_cb = L.Trainer(
    max_epochs=5, callbacks=[ActivationLoggerCallback()], logger=logger
)
trainer_with_custom_cb.fit(model, train_loader, val_loader)


### 🎯 Fazit

Mit **Callbacks** und **TensorBoard** kannst du das Modelltraining nicht nur besser verstehen, sondern auch effizienter und intelligenter steuern:
- **Zeit sparen** durch frühzeitiges Stoppen
- **Beste Modelle sichern**
- **Lernrate adaptiv anpassen**
- **Metriken visuell analysieren** für bessere Entscheidungen

PyTorch Lightning macht das einfach – und deine Experimente werden reproduzierbar und sauber dokumentiert.
